### Run in Google CoLab! (Open in new window or new tab)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m-wessler/nbm-verify/blob/master/notebooks/verify_1Dqpf_dev.ipynb)

In [1]:
import os
import sys
import csv
import nbm_funcs


import numpy as np
import pandas as pd
import xarray as xr

import seaborn as sns
import scipy.stats as scipy
import urllib.request as req
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

***
***
# Configuration
Select 'site' to evaluate, modify 'vsite' if an alternate verification site is preferred<br>
Fixed 'date0' at the start of the NBM v3.2 period (2/20/2020)<br>
Full lead time is 263 hours - Note if date1 is within this period, there will be missing verification data as it does not exist yet!

In [2]:
# NBM 1D Viewer Site to use
# site = sys.argv[1]
site = nbm_funcs._site = 'KSEA'
vsite = site

# Data Range
lead_time_end = 263
init_hours = [13]#[1, 7, 13, 19]

date0 = nbm_funcs._date0 = datetime(2020, 3, 1)
date1 = nbm_funcs._date1 = datetime(2020, 7, 15)

In [3]:
sitepath = site if site == vsite else '_'.join([site, vsite])

datadir = nbm_funcs._datadir = '../archive/%s/data/'%sitepath
os.makedirs(datadir, exist_ok=True)

figdir = nbm_funcs._figdir = '../archive//%s/figures/'%sitepath
os.makedirs(figdir, exist_ok=True)

dates = pd.date_range(date0, date1, freq='1D')
date2 = nbm_funcs._date2 = date1 + timedelta(hours=lead_time_end)

print(('\nForecast Site: {}\nVerif Site: {}\nInit Hours: '+
      '{}\nFirst Init: {}\nLast Init: {}\nLast Verif: {}').format(
    site, vsite, init_hours, date0, date1, date2))


Forecast Site: KSEA
Verif Site: KSEA
Init Hours: [13]
First Init: 2020-03-01 00:00:00
Last Init: 2020-07-15 00:00:00
Last Verif: 2020-07-25 23:00:00


***
***
# Obtain observation data from SynopticLabs (MesoWest) API
These are quality-controlled precipitation observations with adjustable accumulation periods<br>
See more at: https://developers.synopticdata.com/mesonet/v2/stations/precipitation/
<br><br>
If no observation file exists, will download and save for future use

In [4]:
obfile = datadir + '%s_obs_%s_%s.pd'%(site, date0.strftime('%Y%m%d'), date1.strftime('%Y%m%d'))

if os.path.isfile(obfile):
    # Load file
    obs = pd.read_pickle(obfile)
    print('\nLoaded obs from file %s\n'%obfile)

else:
    # Get and save file
    obs = nbm_funcs.get_precip_obs(vsite, date0, date2)
    obs = obs[0].merge(obs[1], how='inner', on='ValidTime').merge(obs[2], how='inner', on='ValidTime')
    obs = obs[[k for k in obs.keys() if 'precip' in k]].sort_index()

    obs.to_pickle(obfile)
    print('\nSaved obs to file %s\n'%obfile)
    
mm_in = 1/25.4
obs *= mm_in
[obs.rename(columns={k:k.replace('mm', 'in')}, inplace=True) for k in obs.keys()]

obs.describe().T


Loaded obs from file ../archive/KSEA/data/KSEA_obs_20200301_20200715.pd



,count,mean,std,min,25%,50%,75%,max
6h_precip_in,578.0,0.018204,0.066480,0.0,0.0,0.0,0.00000,0.64
12h_precip_in,290.0,0.035161,0.107487,0.0,0.0,0.0,0.00873,0.78
24h_precip_in,290.0,0.072874,0.163465,0.0,0.0,0.0,0.05000,0.87


***
***
# Obtain NBM forecast data from NBM 1D Viewer (csv file API)
These are the NBM 1D output files extracted from the viewer with 3 set accumulation periods<br>
See more at: https://hwp-viz.gsd.esrl.noaa.gov/wave1d/?location=KSLC&col=2&hgt=1&obs=true&fontsize=1&selectedgroup=Default
<br><br>
If no forecast file exists, will download and save for future use. This can take some time.

In [5]:
nbmfile = datadir + '%s_nbm_%s_%s.pd'%(site, date0.strftime('%Y%m%d'), date1.strftime('%Y%m%d'))

if os.path.isfile(nbmfile):
    # Load file
    nbm = pd.read_pickle(nbmfile)
    print('Loaded NBM from file %s'%nbmfile)

else:
    url_list = []
    for date in dates:
        for init_hour in init_hours:
            # For now pull from the csv generator
            # Best to get API access or store locally later
            base = 'https://hwp-viz.gsd.esrl.noaa.gov/wave1d/data/archive/'
            datestr = '{:04d}/{:02d}/{:02d}'.format(date.year, date.month, date.day)
            sitestr = '/NBM/{:02d}/{:s}.csv'.format(init_hour, site)
            url_list.append([date, init_hour, base + datestr + sitestr])

    # Try multiprocessing this for speed?
    nbm = np.array([nbm_funcs.get_1d_csv(url, this=i+1, total=len(url_list)) for i, url in enumerate(url_list)])
    nbm = np.array([line for line in nbm if line is not None])

    header = nbm[0, 0]
    
    # This drops days with incomplete collections. There may be some use
    # to keeping this data, can fix in the future if need be
    # May also want to make the 100 value flexible!
    nbm = np.array([np.array(line[1]) for line in nbm if len(line[1]) == 100])

    nbm = nbm.reshape(-1, nbm.shape[-1])
    nbm[np.where(nbm == '')] = np.nan

    # Aggregate to a clean dataframe
    nbm = pd.DataFrame(nbm, columns=header).set_index(
        ['InitTime', 'ValidTime']).sort_index()

    # Drop last column (misc metadata?)
    nbm = nbm.iloc[:, :-2].astype(float)
    header = nbm.columns

    # variables = np.unique([k.split('_')[0] for k in header])
    # levels = np.unique([k.split('_')[1] for k in header])

    init =  nbm.index.get_level_values(0)
    valid = nbm.index.get_level_values(1)

    # Note the 1h 'fudge factor' in the lead time here
    lead = pd.DataFrame(
        np.transpose([init, valid, ((valid - init).values/3600/1e9).astype(int)+1]), 
        columns=['InitTime', 'ValidTime', 'LeadTime']).set_index(['InitTime', 'ValidTime'])

    nbm.insert(0, 'LeadTime', lead)

    klist = np.array([k for k in np.unique([k for k in list(nbm.keys())]) if ('APCP' in k)&('1hr' not in k)])
    klist = klist[np.argsort(klist)]
    klist = np.append('LeadTime', klist)
    nbm = nbm.loc[:, klist]
    
    # Nix values where lead time shorter than acc interval
    for k in nbm.keys():
        if 'APCP24hr' in k:
            nbm[k][nbm['LeadTime'] < 24] = np.nan
        elif 'APCP12hr' in k:
            nbm[k][nbm['LeadTime'] < 12] = np.nan
        elif 'APCP6hr' in k:
            nbm[k][nbm['LeadTime'] < 6] = np.nan
        else:
            pass
    
    nbm.to_pickle(nbmfile)
    print('\nSaved NBM to file %s'%obfile)

# Convert mm to in
nbm = pd.DataFrame([nbm['LeadTime']] + [nbm[k] * mm_in for k in nbm.keys() if 'LeadTime' not in k]).T

# Display some basic stats
nbm.loc[:, ['APCP6hr_surface', 'APCP6hr_surface_70% level', 'APCP6hr_surface_50% level',
            'APCP12hr_surface', 'APCP12hr_surface_70% level', 'APCP12hr_surface_50% level',
            'APCP24hr_surface', 'APCP24hr_surface_70% level', 'APCP24hr_surface_50% level'
            ]].describe().T

Loaded NBM from file ../archive/KSEA/data/KSEA_nbm_20200301_20200715.pd


,count,mean,std,min,25%,50%,75%,max
APCP6hr_surface,5349.0,0.013527,0.037232,0.0,0.0,0.000000,0.000000,0.390000
APCP6hr_surface_70% level,5456.0,0.014361,0.037641,0.0,0.0,0.000000,0.006929,0.417087
APCP6hr_surface_50% level,5456.0,0.004505,0.020444,0.0,0.0,0.000000,0.000000,0.329173
APCP12hr_surface,5332.0,0.034732,0.075785,0.0,0.0,0.000000,0.029331,0.686614
APCP12hr_surface_70% level,5332.0,0.035657,0.070221,0.0,0.0,0.000394,0.040167,0.676181
APCP12hr_surface_50% level,5332.0,0.013262,0.040737,0.0,0.0,0.000000,0.001024,0.490551
APCP24hr_surface,2976.0,0.072128,0.127029,0.0,0.0,0.002008,0.091939,0.789685
APCP24hr_surface_70% level,2976.0,0.085937,0.124312,0.0,0.0,0.027559,0.127608,0.813071
APCP24hr_surface_50% level,2976.0,0.040345,0.081238,0.0,0.0,0.000039,0.042293,0.616417


#### Plot the distribution of precipitation observations vs forecasts for assessment of representativeness

In [6]:
thresh_id = nbm_funcs._thresh_id = {'Small':[0, 1], 'Medium':[1, 2], 'Large':[2, 3], 'All':[0, 3]}

# 33rd, 67th percentile determined above
thresholds = nbm_funcs._thresholds = {interval:nbm_funcs.apcp_dist_plot(obs, nbm, interval) 
              for interval in [6, 12, 24]}

# Use fixed override if desired
# thresholds = {
#     6:[1, 2],
#     12:[1, 2],
#     24:[1, 2]}

thresholds

KSEA_6h.APCP_dist.png
KSEA_12h.APCP_dist.png
KSEA_24h.APCP_dist.png


{6: array([0.  , 0.04, 0.13, 0.64]),
 12: array([0.        , 0.04      , 0.13108268, 0.78      ]),
 24: array([0.  , 0.05, 0.21, 0.87])}

***
***
# Reorganize the data for analysis:
#### Isolate the forecasts by accumulation interval and lead time

In [7]:
plist = np.arange(1, 100)

data = []
for interval in [6, 12, 24]:
    
    pkeys = np.array([k for k in nbm.keys() if '%dhr_'%interval in k])
    pkeys = np.array([k for k in pkeys if '%' in k])
    pkeys = pkeys[np.argsort([int(k.split('_')[-1].split('%')[0]) for k in pkeys])]
    
    for lead_time in np.arange(interval, lead_time_end, 6):
        
        for esize in ['Small', 'Medium', 'Large']:
            
            thresh = [thresholds[interval][thresh_id[esize][0]], 
                      thresholds[interval][thresh_id[esize][1]]]
        
            print('\rProcessing interval %d lead %dh'%(interval, lead_time), end='')

            # We need to break out the verification to each lead time,
            # but within each lead time we have a number of valid times.
            # At each lead time, valid time, isolate the forecast verification

            # Combine the datasets to make it easier to work with
            idata = nbm[nbm['LeadTime'] == lead_time].merge(obs, on='ValidTime').drop(columns='LeadTime')

            # Subset for event size
            iobs = idata['%dh_precip_in'%interval]
            idata = idata[((iobs >= thresh[0]) & (iobs < thresh[1]))]

            for itime in idata.index:

                try:
                    prob_fx = idata.loc[itime, pkeys].values
                    mean_fx = np.nanmean(prob_fx)
                    std_fx = np.nanstd(prob_fx)
                    med_fx = idata.loc[itime, 'APCP%dhr_surface_50%% level'%interval]
                    det_fx = idata.loc[itime, 'APCP%dhr_surface'%interval]

                    # Optional - leave as nan?
                    det_fx = det_fx if ~np.isnan(det_fx) else 0.

                    verif_ob = idata.loc[itime, '%dh_precip_in'%interval]
                    
                    verif_rank = np.searchsorted(prob_fx, verif_ob, 'right')                    
                    verif_rank_val = prob_fx[verif_rank-1]
                    verif_rank_error = verif_rank_val - verif_ob
                    
                    verif_rank = 101 if ((verif_rank >= 99) & (verif_ob > verif_rank_val)) else verif_rank
                    verif_rank = -1 if ((verif_rank <= 1) & (verif_ob < verif_rank_val)) else verif_rank
                    
                    det_rank = np.searchsorted(prob_fx, det_fx, 'right')
                    det_error = det_fx - verif_ob

                except:
                    raise
                    # pass
                    # print('failed', itime)

                else:
                    if ((verif_ob > 0.) & ~np.isnan(verif_rank_val)):

                        data.append([
                            # Indexers
                            interval, lead_time, itime, esize,

                            # Verification and deterministic
                            verif_ob, det_fx, det_rank, det_error,

                            # Probabilistic
                            verif_rank, verif_rank_val, verif_rank_error, 
                            med_fx, mean_fx, std_fx])

data = pd.DataFrame(data, columns=['Interval', 'LeadTime', 'ValidTime', 'EventSize',
                'verif_ob', 'det_fx', 'det_rank', 'det_error',
                'verif_rank', 'verif_rank_val', 'verif_rank_error', 
                'med_fx', 'mean_fx', 'std_fx'])

print('\n\nAvailable keys:\n\t\t{}\nn rows: {}'.format('\n\t\t'.join(data.keys()), len(data)))

Processing interval 24 lead 258h

Available keys:
		Interval
		LeadTime
		ValidTime
		EventSize
		verif_ob
		det_fx
		det_rank
		det_error
		verif_rank
		verif_rank_val
		verif_rank_error
		med_fx
		mean_fx
		std_fx
n rows: 1803


***
***
# Create Bulk Temporal Stats Plots
#### Reliability diagrams, bias over time, rank over time, etc.

#### Plot histograms of percentile rank

In [8]:
short, long = 0, 120
plot_type = 'Verification'
plot_var = 'verif_rank'
esize = 'All'

for interval in [6, 12, 24]:

    kwargs = {'_interval':interval, '_esize':esize,
             '_short':short, '_long':long,
             '_plot_type':plot_type, '_plot_var':plot_var}
    
    nbm_funcs.histograms_verif_rank(data, **kwargs)

KSEA_6h_szAll_lead0-120h.rankPDF.verification.png
KSEA_12h_szAll_lead0-120h.rankPDF.verification.png
KSEA_24h_szAll_lead0-120h.rankPDF.verification.png


#### Plot a reliability diagram style CDF to evaluate percentile rankings

In [9]:
short, long = 0, 120
plot_type = 'Verification'
plot_var = 'verif_rank'
esize = 'All'

for interval in [6, 12, 24]:

    kwargs = {'_interval':interval, '_esize':esize,
             '_short':short, '_long':long,
             '_plot_type':plot_type, '_plot_var':plot_var}

    nbm_funcs.reliability_verif_cdf(data, **kwargs)

KSEA_6h_szAll_lead0-120h.reliabilityCDF.verification.png
KSEA_6h_szAll_lead0-120h.reliabilityCDF.verification.png
KSEA_12h_szAll_lead0-120h.reliabilityCDF.verification.png
KSEA_12h_szAll_lead0-120h.reliabilityCDF.verification.png
KSEA_24h_szAll_lead0-120h.reliabilityCDF.verification.png
KSEA_24h_szAll_lead0-120h.reliabilityCDF.verification.png


#### Produce bias, ME, MAE, and percentile rank plots as they evolve over time
This helps illustrate at what leads a dry/wet bias may exist and how severe it may be<br>
Adds value in interpreting the CDF reliability diagrams

In [10]:
short, long = 0, 120
esize = 'All'

for interval in [6, 12, 24]:

    kwargs = {'_interval':interval, '_esize':esize,
             '_short':short, '_long':long}

    nbm_funcs.rank_over_leadtime(data, **kwargs)

KSEA_6h_szAll_lead0-120h.rank_err_overtime.png
KSEA_6h_szAll_lead0-120h.rank_err_overtime.png
KSEA_12h_szAll_lead0-120h.rank_err_overtime.png
KSEA_12h_szAll_lead0-120h.rank_err_overtime.png
KSEA_24h_szAll_lead0-120h.rank_err_overtime.png
KSEA_24h_szAll_lead0-120h.rank_err_overtime.png


## 